In [1]:
!pip install dataframe_image

In [7]:
LOCAL=True

if LOCAL == False:
   stub = modal.Stub()
   hopsworks_image = modal.Image.debian_slim().pip_install(["hopsworks","joblib","seaborn","sklearn==1.1.1","dataframe-image","yfinance"])
   @stub.function(image=hopsworks_image, schedule=modal.Period(days=1), secret=modal.Secret.from_name("HOPSWORKS_API_KEY"))
   def f():
       g()

def g():
    import pandas as pd
    import hopsworks
    import joblib
    import datetime
    from PIL import Image
    from datetime import datetime
    import dataframe_image as dfi
    from sklearn.metrics import confusion_matrix
    from matplotlib import pyplot
    import seaborn as sns
    import requests

    project = hopsworks.login()
    fs = project.get_feature_store()
    
    mr = project.get_model_registry()
    model = mr.get_model("sp500_model", version=1)
    model_dir = model.download()
    model = joblib.load(model_dir + "/sp500_model.pkl")
    
    feature_view = fs.get_feature_view(name="sp500", version=1)
    batch_data = feature_view.get_batch_data() 
    
    
    y_pred = model.predict(batch_data)
    print(y_pred)
    offset = 1
    target = y_pred[y_pred.size-offset]
    #target_url = "https://raw.githubusercontent.com/Deepak0197/id2223_Project_sp500_stocks/main/sp500_dataset/" + str(int(target)) + ".png"
    print("sp500 stock predicted: " + str(target))

    dataset_api = project.get_dataset_api() 
    sp500_fg = fs.get_feature_group(name="sp500", version=1)
    df = sp500_fg.read() 
    #print(df)
    label = df.iloc[-offset]["close"]
    #label_url = "https://raw.githubusercontent.com/Deepak0197/id2223_Project_sp500_stocks/main/sp500_dataset/" + str(int(label)) + ".png"
    print("sp500 actual stock: " + str(label))
    #img = Image.open(requests.get(label_url, stream=True).raw)            
    #img.save("./actual_sp500.png")
    #dataset_api.upload("./actual_sp500.png", "Resources/images", overwrite=True)
    
    monitor_fg = fs.get_or_create_feature_group(name="sp500_direction_predictions",
                                                version=1,
                                                primary_key=["datetime"],
                                                description="Sp500 Stock Direction predictions Monitoring"
                                                )
    
    now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    data = {
        'prediction': [target],
        'label': [label],
        'datetime': [now],
       }
    monitor_df = pd.DataFrame(data)
    monitor_fg.insert(monitor_df, write_options={"wait_for_job" : False})
    
    history_df = monitor_fg.read()
    # Add our prediction to the history, as the history_df won't have it - 
    # the insertion was done asynchronously, so it will take ~1 min to land on App
    history_df = pd.concat([history_df, monitor_df])


    df_recent = history_df.tail(4)
    dfi.export(df_recent, './df_recent.png', table_conversion = 'matplotlib')
    dataset_api.upload("./df_recent.png", "Resources/images", overwrite=True)
    
    predictions = history_df[['prediction']]
    labels = history_df[['label']]
    
    print("history_df: ", history_df)


if __name__ == "__main__":
    if LOCAL == True :
        g()
    else:
        with stub.run():
            f()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/210071
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (1.35s) 
[[1177.39189409]
 [1228.61399542]
 [1325.07612612]
 ...
 [1097.86308319]
 [1249.79464002]
 [1191.87415308]]
sp500 stock predicted: [1191.87415308]


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Finished: Reading data from Hopsworks, using ArrowFlight (1.01s) 
sp500 actual stock: 1194.0699462890625


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: sp500_direction_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/210071/jobs/named/sp500_direction_predictions_1_offline_fg_materialization/executions
Finished: Reading data from Hopsworks, using ArrowFlight (0.78s) 


Uploading: 0.000%|          | 0/43757 elapsed<00:00 remaining<?

history_df:               prediction        label              datetime
0  [1191.8741530798077]  1194.069946  01/07/2024, 15:53:09
1  [1191.8741530798077]  1194.069946  01/07/2024, 15:53:43
2  [1191.8741530798077]  1194.069946  01/07/2024, 15:55:28
3  [1191.8741530798077]  1194.069946  01/07/2024, 15:54:15
0  [1191.8741530798077]  1194.069946  01/07/2024, 15:58:51
